In [72]:

import os, re, sys
import numpy as np

filename = "data.txt"
with open(filename, 'r') as f:
    s = f.read()
    initial, compuatations = s.split("\n\n")


calculated = {x.split(": ")[0]: bool(int(x.split(": ")[1])) for x in initial.split("\n")}
compuatations = compuatations.split("\n")



outcomes = []
dependencies = []
operations = []

for compuatation in compuatations:
    args, outcome = compuatation.split(" -> ")
    args = args.split(" ")

    dependencies.append((args[0], args[2]))
    operations.append(args[1])
    outcomes.append(outcome)



#print(outcomes)


In [ ]:


def get_dependencies(name):
    deps = set()
    for i in range(len(outcomes)):
        if outcomes[i] == name:
            a, b = dependencies[i] 
            

            #print(name + " is  the result of " + a + " "+ operations[i] + " " + b)

            if a[0] in ['x', 'y']:
                deps.add(a)
            else:
                deps |= get_dependencies(a)


            if b[0] in ['x', 'y']:
                deps.add(b)
            else:
                deps |= get_dependencies(b)

            return deps
    return deps


def get_influences_below_i(name, max_i): # depends on get_dependencies and dependencies and outcomes
    infs = set()
    for i in range(len(dependencies)):
        a, b = dependencies[i]
        c = outcomes[i]

        if name == a:
            deps = get_dependencies(b) | {b}

            for dep in deps:
                if dep[0] in ['x', 'y'] and int(dep[1:]) > max_i:
                    break # b depends in some way on an x and y bigger than max_i -> dont use it
            else:
                infs = infs | {c} | get_influences_below_i(c, max_i)

        if name == b:
            deps = get_dependencies(a) | {a}

            for dep in deps:
                if dep[0] in ['x', 'y'] and int(dep[1:]) > max_i:
                    break # b depends in some way on an x and y bigger than max_i -> dont use it
            else:
                infs = infs | {c} | get_influences_below_i(c, max_i)
    
    return infs
#print(get_dependencies('x01'))

print(get_influences_below_i('x01', 1))
print(get_influences_below_i('y01', 1))



{'hth', 'kjn', 'z01', 'pgc', 'hjc'}
{'hth', 'kjn', 'z01', 'pgc', 'hjc'}


In [74]:

affects = {}
for i in range(len(dependencies)):
    a, b = dependencies[i]
    op = operations[i]
    c = outcomes[i]
    if affects.get(a):
        affects[a].append(c)
    else:
        affects[a] = [c] 

    if affects.get(b):
        affects[b].append(c)
    else:
        affects[b] = [c] 


def comp(a, b, op):
    if op == 'XOR':
        return a ^ b
    elif op == 'OR':
        return a | b
    elif op == 'AND':
        return a & b


def compute(local_results, pos): # uses dependencies
    prev_l, l = -1, 0
    while l > prev_l:
        prev_l = l
        if pos in local_results:
            return local_results[pos]

        for i in range(len(dependencies)):
            a, b = dependencies[i]
            if a in local_results and b in local_results:
                a, b = local_results[a], local_results[b]
                local_results[outcomes[i]] = comp(a, b, operations[i])
        
        l = len(local_results)

        
def find_carry_and_z(x, y, c, possible_cz): # uses compute
    poss_start_values = [ \
        {x: 0, y: 0, c:0}, 
        {x: 0, y: 0, c:1},
        {x: 0, y: 1, c:0},
        {x: 0, y: 1, c:1},
        {x: 1, y: 0, c:0},
        {x: 1, y: 0, c:1},
        {x: 1, y: 1, c:0},
        {x: 1, y: 1, c:1},
    ]
    corr_results_c = [
        0, 0, 0, 1, 0, 1, 1, 1
    ]
    corr_results_z = [
        0, 1, 1, 0, 1, 0, 0, 1
    ]

    poss_carry_bits = []
    poss_outc_bits = []
    for cz in possible_cz:
        ress = []
        for i in range(8):
            start_values = poss_start_values[i].copy()
            ress.append(compute(start_values, cz))
            
        if ress == corr_results_c:
            poss_carry_bits.append(cz)
        elif ress == corr_results_z:
            poss_outc_bits.append(cz)

    return poss_carry_bits, poss_outc_bits



In [75]:

def swap(x, y):
    if y in affects.get(x, []) or x in affects.get(y, []):
        return # it is impossible to swap these


    x_i = None
    y_i = None
    for i in range(len(dependencies)):
        a, b = dependencies[i]
        c = outcomes[i]

        if x == c:
            x_i = i
            affects[a].remove(x)
            affects[b].remove(x)
            affects[a].append(y)
            affects[b].append(y)

        if y == c:
            y_i = i
            affects[a].remove(y)
            affects[b].remove(y)
            affects[a].append(x)
            affects[b].append(x)
    
    outcomes[x_i] = y
    outcomes[y_i] = x





seen_elems = set()


outputs = ["z00"]
carries = ["skt"]
prev_carry = "skt" # the first carry bit
all_swaps = []
for i in range(1, 45):
    x, y, z = (char + ["0", ''][i >= 10] + str(i) for char in ['x', 'y', 'z'])
    
    inf_x = get_influences_below_i(x, i)
    inf_y = get_influences_below_i(y, i)

    poss_cz = (inf_x | inf_y) - seen_elems # all the elements that depend solely on x00, x01, y00 and y01
    seen_elems = seen_elems | inf_x | inf_y
    
    c_poss, z_poss = find_carry_and_z(x, y, carries[-1], poss_cz)

    if z in z_poss and len(c_poss) >= 1:
        carries.append(c_poss[-1])
        outputs.append(z)
    else:
  
        solved = False
        poss_cz = list(poss_cz)

        for i in range(len(poss_cz)):
            cz1 = poss_cz[i]
            for j in range(i+1, len(poss_cz)):
                cz2 = poss_cz[j]

                swap(cz1, cz2)
                c_poss, z_poss = find_carry_and_z(x, y, carries[-1], poss_cz)

                if z in z_poss and len(c_poss) >= 1:
                    carries.append(c_poss[-1])
                    outputs.append(z)
                    all_swaps.append(cz1)
                    all_swaps.append(cz2)
                    solved = True
                
                swap(cz1, cz2) # swap them back
                
            
            if solved:
                break




In [76]:
print(all_swaps)

print(",".join(sorted(all_swaps)))

['z06', 'fkp', 'ngr', 'z11', 'z31', 'mfm', 'krj', 'bpt']
bpt,fkp,krj,mfm,ngr,z06,z11,z31
